In [1]:
import pandas as pd

data = pd.read_csv('./data/train.csv')

data.head()
# Pclass, Sex, Age, SibSp, Parch, Fare

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
import numpy as np

interested_columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']

data = data[interested_columns]
data.dropna(inplace=True)

data['Sex'] = data['Sex'] == 'male'
data['Sex'] = np.array(data['Sex'], dtype=np.int32)

data.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,2.236695,0.634454,29.699118,0.512605,0.431373,34.694514,0.406162
std,0.838250,0.481921,14.526497,0.929783,0.853289,52.918930,0.491460
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,20.125000,0.000000,0.000000,8.050000,0.000000
50%,2.000000,1.000000,28.000000,0.000000,0.000000,15.741700,0.000000
75%,3.000000,1.000000,38.000000,1.000000,1.000000,33.375000,1.000000
max,3.000000,1.000000,80.000000,5.000000,6.000000,512.329200,1.000000


In [3]:
# normalize data to be between 0 and 1

for col in interested_columns:
    data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())

In [4]:
data.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,0.618347,0.634454,0.367921,0.102521,0.071895,0.067719,0.406162
std,0.419125,0.481921,0.182540,0.185957,0.142215,0.103291,0.491460
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.247612,0.000000,0.000000,0.015713,0.000000
50%,0.500000,1.000000,0.346569,0.000000,0.000000,0.030726,0.000000
75%,1.000000,1.000000,0.472229,0.200000,0.166667,0.065144,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
data = np.array(data)

X, y = data[:,:-1], data[:,-1]

# add a column of ones to x
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

print(X.shape, y.shape)

(714, 7) (714,)


### Loss function

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

class LossFunction:

    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def loss(self, a : np.ndarray):
        prob = sigmoid(np.dot(self.X, a))
        values = - self.y * np.log(prob) - (1 - self.y) * np.log(1 - prob)
        return np.nansum(values)

    def gradient(self, a : np.ndarray):
        prob = sigmoid(np.dot(self.X, a))
        sub_coefficient = -(self.y - prob) 
        return np.dot(self.X.T, sub_coefficient)
    
    def precision(self, a : np.ndarray):
        prob = sigmoid(np.dot(self.X, a))
        prob = np.array(prob >= 0.5, dtype=np.int32)
        return np.sum(prob == self.y) / self.y.shape[0]

loss_func = LossFunction(X, y)


In [7]:
def gradient_descent(loss_func, starting_point, learning_rate = 0.001, num_steps = 100, precision=0.00001):
    cur_point = starting_point
    for i in range(num_steps):
        grad = loss_func.gradient(cur_point)
        print("Iteration {}: loss = {}, precision = {}".format(i, loss_func.loss(cur_point), loss_func.precision(cur_point)))
        cur_point = cur_point - learning_rate * grad
        if np.linalg.norm(grad) < precision:
            break
    return cur_point

optimal = gradient_descent(loss_func, np.zeros(X.shape[1]))

Iteration 0: loss = 494.9070869198009, precision = 0.4061624649859944
Iteration 1: loss = 467.6783802061075, precision = 0.5938375350140056
Iteration 2: loss = 453.6741382771319, precision = 0.5938375350140056
Iteration 3: loss = 445.1058564352742, precision = 0.5938375350140056
Iteration 4: loss = 438.90394764942016, precision = 0.5938375350140056
Iteration 5: loss = 433.83170212259313, precision = 0.5938375350140056
Iteration 6: loss = 429.3701530356671, precision = 0.5938375350140056
Iteration 7: loss = 425.2914007329867, precision = 0.6050420168067226
Iteration 8: loss = 421.48983233889584, precision = 0.7016806722689075
Iteration 9: loss = 417.912796770911, precision = 0.7044817927170869
Iteration 10: loss = 414.5312058332657, precision = 0.7044817927170869
Iteration 11: loss = 411.32675329111805, precision = 0.7044817927170869
Iteration 12: loss = 408.2862526917245, precision = 0.7044817927170869
Iteration 13: loss = 405.39909167748783, precision = 0.7044817927170869
Iteration 14